In [1]:
import pandas as pd

In [2]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")

In [3]:
column = ['занятость', 'по должности-лемме', 'по дополнительному признаку', 'по условиям', 'общие фразы']

In [4]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()
#for i in 'работа финансовым директором'.split():
#    print(morph.normal_forms(i))
from nltk.corpus import stopwords
stopwords_ru = stopwords.words("russian")

In [5]:
def morphe(str):
    return ' '.join([morph.normal_forms(w)[0] for w in str.split() if w not in stopwords_ru])

In [6]:
df = pd.read_csv("answers.csv")
df = df[["query", *column]]
#df = df[df["по должности-лемме"].notna()]
for col in column:
    df.fillna({col: 'Нет'}, inplace=True)

df['stem_query'] = df['query'].apply(morphe)
df

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы,stem_query
0,фарпост работа владивосток,Нет,Нет,Нет,Нет,общая фраза,фарпост работа владивосток
1,кофейни вакансии,Нет,Нет,Нет,Нет,общая фраза,кофейня вакансия
2,работа разнорабочие часовой,Нет,Рабочий,Нет,Нет,Нет,работа разнорабочий часовой
3,личный водитель на день,на неполный день,Водитель,Нет,Нет,Нет,личный водитель день
4,работа от работодателя персональный водитель,Нет,Водитель,Нет,Нет,Нет,работа работодатель персональный водитель
...,...,...,...,...,...,...,...
14240,япония вакансии,Нет,Нет,Нет,Нет,общая фраза,япония вакансия
14241,япония работа,Нет,Нет,Нет,Нет,общая фраза,япония работа
14242,японский язык работа,Нет,Нет,Нет,Нет,общая фраза,японский язык работа
14243,яппи вакансии,Нет,Нет,Нет,Нет,общая фраза,яппи вакансия


In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

def split(x):
    return [i for i in set(x.split(',')) if len(i)>0]

df['занятость'] = df['занятость'].apply(lambda x: split(x))
df['по дополнительному признаку'] = df['по дополнительному признаку'].apply(lambda x: x.split(','))

mlb = MultiLabelBinarizer()
df_encoded = pd.DataFrame(mlb.fit_transform(df['занятость']), columns=mlb.classes_)
df = df.join(df_encoded)

mlb2 = MultiLabelBinarizer()
df_encoded2 = pd.DataFrame(mlb2.fit_transform(df['по дополнительному признаку']), columns=mlb2.classes_)
df = df.join(df_encoded2, lsuffix='_занятость', rsuffix='_доп')

In [8]:
df

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы,stem_query,Вахта,Вечерняя,Временная,...,на неполный день,по выходным,Нет_доп,без опыта,для женщин,для инвалидов,для мужчин,для пенсионеров,для студентов,для школьников
0,фарпост работа владивосток,[Нет],Нет,[Нет],Нет,общая фраза,фарпост работа владивосток,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,кофейни вакансии,[Нет],Нет,[Нет],Нет,общая фраза,кофейня вакансия,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,работа разнорабочие часовой,[Нет],Рабочий,[Нет],Нет,Нет,работа разнорабочий часовой,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,личный водитель на день,[на неполный день],Водитель,[Нет],Нет,Нет,личный водитель день,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,работа от работодателя персональный водитель,[Нет],Водитель,[Нет],Нет,Нет,работа работодатель персональный водитель,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14240,япония вакансии,[Нет],Нет,[Нет],Нет,общая фраза,япония вакансия,0,0,0,...,0,0,1,0,0,0,0,0,0,0
14241,япония работа,[Нет],Нет,[Нет],Нет,общая фраза,япония работа,0,0,0,...,0,0,1,0,0,0,0,0,0,0
14242,японский язык работа,[Нет],Нет,[Нет],Нет,общая фраза,японский язык работа,0,0,0,...,0,0,1,0,0,0,0,0,0,0
14243,яппи вакансии,[Нет],Нет,[Нет],Нет,общая фраза,яппи вакансия,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split

X = df['stem_query'].values
y = df.drop(columns=['query', 'занятость', 'по дополнительному признаку', 'stem_query']).values

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

query_train, query_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

In [10]:
df.drop(columns=['query', 'занятость', 'по дополнительному признаку', 'stem_query']).columns

Index(['по должности-лемме', 'по условиям', 'общие фразы', 'Вахта', 'Вечерняя',
       'Временная', 'Дневная', 'Нет_занятость', 'Ночная', 'Подработка',
       'Посменная', 'Посуточная', 'Удаленная', 'на дому', 'на неполный день',
       'по выходным', 'Нет_доп', 'без опыта', 'для женщин', 'для инвалидов',
       'для мужчин', 'для пенсионеров', 'для студентов', 'для школьников'],
      dtype='object')

In [11]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

encoder.fit(y)

y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X)

X_train = vectorizer.transform(query_train)
X_test  = vectorizer.transform(query_test)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(max_iter=400)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.9432902863559798


In [14]:
extra = df.drop(columns=['query', 'занятость', 'по дополнительному признаку', 'stem_query']).columns.tolist()

In [15]:
def predict_position(query):
    query = morphe(query)
    query_transformed = vectorizer.transform([query])
    prediction = classifier.predict(query_transformed)
    return encoder.inverse_transform(prediction)[0]

example_query = "работа во владивостоке"
predicted_position = predict_position(example_query)
predicted_position

# for i in range(len(extra)):
#     print(f"Предсказание {extra[i]}", predicted_position[i])

array(['Нет', 'Нет', 'общая фраза', 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0], dtype=object)